In [ ]:
import pandas as pd


In [ ]:
gsheetid='1YQB5F4Tx0lmgSRQfmUAWeK9rMMbgXOicKl6gKG42SpY'
sheet_name='Legal_dataset'

In [ ]:
gsheet_url = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(gsheetid, sheet_name)
gsheet_url

In [ ]:
df=pd.read_csv(gsheet_url)
column_name = 'Question'
df.dropna(subset=[column_name], inplace=True)
df1=df.iloc[:, [1, 4, 5]]
ds = df1.dropna(how='all')
ds=ds.rename(columns={'Thématique':'Thematique','Type de droit':'Type_de_droit'})


In [ ]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai.api_key.replace("<", "").replace(">", "")


In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo-0613",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
def evaluation(x):
    delimiter = "####"
    system_message = f"""
    Étant donné le message d'un client d'une entreprise juridique,tu dois sélectionnez la categorie 
    la mieux adaptée à l'entrée. 
    le message du client sera delimité par {delimiter} 

    Vous recevrez les noms des categories 
    disponibles et une description de celle qui convient le mieux à l'entrée. 

    Renseignement_juridique =  une connaissance approfondie des lois, des réglementations et des meilleures pratiques en matière
    de renseignement. Ce domaine implique la capacité de fournir des conseils juridiques et une orientation aux entreprises sur une
    variété de sujets, tels que la gouvernance d'entreprise, les contrats, les fusions et acquisitions, la conformité et le règlement des litiges.
    Les professionnels de ce domaine possèdent d'excellentes compétences 
    analytiques et de résolution de problèmes, et ils sont capables de naviguer efficacement dans des cadres juridiques complexes



    Banque = une connaissance approfondie des lois et réglementations bancaires, ainsi que des meilleures pratiques du secteur. 
    Ce domaine implique une vaste expérience dans l'assistance aux clients en matière de services bancaires tels que l'ouverture de comptes, 
    les prêts, les investissements et la gestion des risques financiers. Les professionnels de ce domaine sont compétents pour fournir des conseils en matière de planification financière, 
    de gestion de trésorerie, de conformité réglementaire et de stratégies de croissance financière.
    Ils comprennent les besoins spécifiques de chaque client et proposent des solutions bancaires personnalisées pour les aider à atteindre leurs objectifs.



    Domiciliation = une connaissance approfondie des lois, réglementations et pratiques courantes en matière de domiciliation et de comptabilité. 
    Ce domaine implique une excellente compétence dans l'assistance et le soutien aux entreprises en matière de domiciliation, 
    de création de sociétés, de tenue de livres comptables et de déclarations fiscales. 
    Les professionnels de ce domaine ont une expertise dans l'établissement de contrats de location et de baux commerciaux, 
    ainsi que dans la gestion des obligations légales et administratives liées à la domiciliation d'entreprises. 
    Ils fournissent des conseils précieux et une expertise stratégique à leurs clients pour les aider à atteindre leurs objectifs commerciaux en matière de domiciliation et de comptabilité.


    Comptabilite = une connaissance approfondie des principes comptables, de fiscalité, de choix de fiscalité, 
    des normes financières et des meilleures pratiques en matière de comptabilité. Ce domaine implique une expérience étendue dans la tenue des livres comptables,
    l'établissement des états financiers, l'analyse des coûts et la gestion fiscale. Les professionnels de ce domaine sont compétents pour fournir des conseils en matière de gestion financière, 
    d'optimisation fiscale et de conformité réglementaire aux entreprises et aux particuliers.
    Ils sont capables de comprendre les besoins financiers spécifiques de chaque client et de fournir des solutions personnalisées pour les aider à atteindre leurs objectifs financiers.

    RAPPEL : la reponse DOIT être l'un des noms de consigne candidats spécifiés ci-dessus

    voici quelques exemples de message et de la reponse demandée:


    message=Je suis en train de créer ma holding, et je voudrais que la SASU que je possède déjà soit 100% détenue par la holding.
    Mais dans le formulaire de création de la holding, je n'ai pas vu de point me permettant de saisir cette information. Comment faire?
    reponse=Renseignement juridique

    message=Bonjour, dans le cadre de l'ouverture de ma SASU, quels sont le domaine d'activité et les activités à sélectionner ? Je suis consultante en marketing et communication. Je vais également proposer des formations et des audit flash via mon site e commerce -> possibilité de commander un service et de l'acheter en ligne.
    reponse=Renseignement juridique

    message=Bonjour, j'aurai besoin d'un Kbis à jour. Pouvez vouez m'aider à m'en procurer un ? Merci
    reponse=Renseignement juridique
 
    message= mutuelle prendre en tant que gérant non associé de la EURL, est ce assimilé salarié ou travailleur non salariés car sur certains sites, il est mentionné que gérant non associé est (assimilé salarié), merci d'avance
    reponse=Renseignement juridique

    message=quelles démarches à réaliser auprès de l'URSSAF
    reponse=Renseignement juridique

    message=Bonjour, pour une SARL , l'associé minoritaire doit il obligatoirement être salarié dans les premiers mois de la création d'entreprise ?
    reponse=Comptabilité

    message=SCI à l'IS assujettie ou non à la TVA ?
    reponse=Comptabilité

    message=Je veux changer de banque pour mon compte pro et déposer le capital social et les statuts d l'entreprise dans une autre banque
    reponse=Banque

    message= Bonjour, je souhaite changer le siège sociale de ma SARL. Actuellement domicilié dans le 92 à Sèvres, je compte déménager dans mon futur local. J'ai signé une promesse de vente sur ce local et j'ai une autorisation de domiciliation du propriétaire (qui détient le local en SCI). Je souhaite savoir cela est possible avant que je ne sois définitivement propriétaire ou si je dois attente d'être propriétaire?
    reponse=Domiciliation

    

    """
    user_message=x.Question	
    messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
    response = get_completion_from_messages(messages)
    evaluation= (response == x.Thematique)
    return evaluation

In [ ]:
ds_test=ds.iloc[1:25]
ds_test

In [ ]:
ds_test.apply(evaluation,axis=1)